# Modelling the AGN continuum
---


In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [4]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


### Defining the AGN continuum
---
In this work, we approximate the AGN continuum as a simple power law,

$$F_{\rm AGN}(\lambda) = F_0 \lambda^{-\alpha_{\lambda}}$$

where the exponent $\alpha_{\lambda} = 2 -\alpha_{\nu}$ where $\alpha_{\nu}$ can range from 1.2 - 2.0 (Groves et al. 2004). 

To determine reasonable values for $F_0$, we use the tight correlation between the total H$\beta$ luminosity, $L_{\rm H\beta}$, and $L_{\rm NT}$, which is defined as the total luminosity of the AGN continuum between 3000 Å and 9500 Å (Neugebauer et al. 1979, Yee 1980), i.e.,

$$L_{\rm NT} = \int^{9500\,Å}_{3000\,Å} F_{\rm AGN}(\lambda) d\lambda 
= \int^{9500\,Å}_{3000\,Å} F_0 \lambda^{-\alpha_{\lambda}} d\lambda 
= \frac{F_0}{1 - \alpha_\lambda}\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)$$

where $F_{\rm AGN}(\lambda)$ is the AGN continuum. In a sample of AGN including quasars, broad & narrow-line radio galaxies and Seyfert 1 & 2 galaxies, Yee (1980) observed a tight correlation such that 

$$L_{\rm NT} \approx 80 L_{\rm H\beta}$$

*Note: equation from [Peterson textbook. eqn. 5.53, p. 90](https://books.google.com.au/books?id=ok4EFlPMStwC&pg=PA90&lpg=PA90&dq=Lnt+agn+continuum&source=bl&ots=QfVvXob4vM&sig=ACfU3U0x69gKrkN-lALkIu0EROAUh1-1vw&hl=en&sa=X&ved=2ahUKEwjW_8fPvqv1AhWXTWwGHarhALcQ6AF6BAgXEAM#v=onepage&q=Lnt%20agn%20continuum&f=false).* 
Referring to table 2 of Dopita et al. (2015), $\log_{10} L_{\rm H\beta} \sim 39 - 42$ in the NLRs of S7 galaxies.
We therefore use our adopted $L_{\rm H\beta}$ in the emission lines to constrain the strength of the AGN continuum in our mock spectra, so that 

$$ F_0 = \frac{80 L_{\rm H\beta}({1 - \alpha_\lambda})}{\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)} $$



In [7]:
###########################################################################
# Defining the AGN continuum using the method of Yee 1980
###########################################################################
L_NT = 1e42

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
for alpha_nu in [0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    L_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    L_lambda = L_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, L_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L_{\rm NT} = 10^{41}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L_{\\rm NT} = 10^{41}\\,\\rm erg\\,s^{-1}$)')

In [8]:
###########################################################################
# Defining the AGN continuum using the method of Heckman+2005
###########################################################################
L_OIII = 1e40  # erg s^-1 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
for alpha_nu in [0.3, 0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    L_5000 = 320 / 5000 * L_OIII  # where L_5000 has units of erg s^-1 Å^-1
    L_lambda_0 = L_5000 / 5000**(-alpha_lambda)

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    L_lambda = L_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, L_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L(\rm [O\,III]) = 10^{40}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L(\\rm [O\\,III]) = 10^{40}\\,\\rm erg\\,s^{-1}$)')

In [9]:
plt.close("all")

In [7]:
###########################################################################
# How do these values compare in the S7 sample?
###########################################################################
plt.close("all")

# Load DataFrame containing Hbeta and [O III] fluxes for the S7 sample
df_s7 = pd.read_csv("/home/u5708159/python/Modules/ppxftests/s7_total_line_fluxes.csv")
df_s7 = df_s7.set_index("Unnamed: 0")
gals = df_s7.index.values

# Compute nonthermal continuum parameters for both Yee 1980 and Heckman+2004
df_s7["L_NT (Yee 1980)"] = df_s7["L_HBETA (total)"] * 80
df_s7["L_lambda_5000 (Heckman+2004)"] = df_s7["L_OIII5007 (total)"] * 320 / 5000

# Compute L_lambda_5000 for the Yee 1980 nonthermal continuum, so we can compare directly with Heckman+2004
alpha_nu = 2.0
alpha_lambda = 2 - alpha_nu
F_lambda_0 = df_s7["L_NT (Yee 1980)"] * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))
df_s7["L_lambda_5000 (Yee 1980)"] = F_lambda_0 * 5000**(-alpha_lambda)

# Compute logs
df_s7["log L_NT (Yee 1980)"] = np.log10(df_s7["L_NT (Yee 1980)"] )
df_s7["log L_lambda_5000 (Yee 1980)"] = np.log10(df_s7["L_lambda_5000 (Yee 1980)"] )
df_s7["log L_lambda_5000 (Heckman+2004)"] = np.log10(df_s7["L_lambda_5000 (Heckman+2004)"])

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
# Plot: [OIII] vs. Hbeta
fig, ax = plt.subplots()
ax.scatter(x=df_s7["log L_HBETA (total)"], y=df_s7["log L_OIII5007 (total)"])
ax.plot([0, 50], [0, 50], "k")
ax.set_xlim([37.5, 43])
ax.set_ylim([37.5, 43])
ax.set_xlabel("log L_HBETA (total)")
ax.set_ylabel("log L_OIII5007 (total)")
ax.grid()

# Plot: L_lambda_5000 for both the Yee 1980 and Heckman+2005
fig, ax = plt.subplots()
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Yee 1980)"].values, label="log L_lambda_5000 (Yee 1980)")
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Heckman+2004)"].values, label="log L_lambda_5000 (Heckman+2004)")
ax.legend()
ax.grid()
ax.set_xlabel("S7 galaxy #")

# Plot: difference in L_lambda_5000 between the Yee 1980 and Heckman+2005
fig, ax = plt.subplots()
ax.scatter(x=range(len(gals)), y=df_s7["log L_lambda_5000 (Heckman+2004)"].values - df_s7["log L_lambda_5000 (Yee 1980)"].values, label="log L_lambda_5000 (Heckman+2004) - log L_lambda_5000 (Yee 1980)")
ax.axhline(0, color="k")
ax.legend()
ax.grid()
ax.set_xlabel("S7 galaxy #")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'S7 galaxy #')

In [9]:
# Functions for evaluating AGN continua

def agn_cont_Y80(lambda_vals_A, L_Hbeta, alpha_nu):
    alpha_lambda = 2 - alpha_nu
    
    # Compute the continuum normalisation
    L_NT = 80 * L_Hbeta
    L_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    return L_lambda_0 * lambda_vals_A**(-alpha_lambda)

def agn_cont_H04(lambda_vals_A, L_OIII, alpha_nu):
    alpha_lambda = 2 - alpha_nu
    
    # Compute the continuum normalisation
    L_5000 = 320 / 5000 * L_OIII  # where L_5000 has units of erg s^-1 Å^-1
    L_lambda_0 = L_5000 / 5000**(-alpha_lambda)

    # Compute the continuum
    return L_lambda_0 * lambda_vals_A**(-alpha_lambda)

In [31]:
gg = 0

In [61]:
# For each S7 galaxy, plot the extracted continuum (in erg s^-1) overlaid with the AGN continuum. 
# This should indicate how reasonable our estimate of the strength of the AGN continuum will be.
plt.close("all")
fig, ax = plt.subplots(figsize=(12, 5))

data_dir = os.environ["S7_DIR"]
fits_path = os.path.join(data_dir, "5_Full_field_spectra")

alpha_nu = 2.0

# Open the FITS file containing the extracted spectrum
gal = gals[gg]
hdulist = fits.open(os.path.join(fits_path, f"{gal}_COMB_full_field.fits"))
spec = hdulist[0].data
spec_err = hdulist[1].data
z = hdulist[0].header["Z"]
lambda_vals_A = (np.array(range(hdulist[0].header["NAXIS1"])) * hdulist[0].header["CDELT1"]) + hdulist[0].header["CRVAL1"]

# Compute AGN spectra

# Plot
ax.clear()
ax.errorbar(x=lambda_vals_A / (1 + z), y=spec, yerr=spec_err, color="k")
# ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, color="r")
ax.plot(lambda_vals_A, agn_cont_Y80(lambda_vals_A, df_s7.loc[gal, "L_HBETA (total)"], alpha_nu=0.3), label=r"Yee 1980 ($\alpha_\nu = %.1f$)" % 0.3, ls="--", color="fuchsia")
ax.plot(lambda_vals_A, agn_cont_H04(lambda_vals_A, df_s7.loc[gal, "L_OIII5007 (total)"], alpha_nu=0.3), label=r"Heckman+2004 ($\alpha_\nu = %.1f$)" % 0.3, ls="--", color="green")
ax.plot(lambda_vals_A, agn_cont_Y80(lambda_vals_A, df_s7.loc[gal, "L_HBETA (total)"], alpha_nu=2.0), label=r"Yee 1980 ($\alpha_\nu = %.1f$)" % 2.0, ls="-", color="fuchsia")
ax.plot(lambda_vals_A, agn_cont_H04(lambda_vals_A, df_s7.loc[gal, "L_OIII5007 (total)"], alpha_nu=2.0), label=r"Heckman+2004 ($\alpha_\nu = %.1f$)" % 2.0, ls="-", color="green")
ax.grid()
ax.axhline(0, ls="--", color="grey")
ax.autoscale(axis="x", tight=True, enable=True)
ax.set_title(gal)
ax.legend()

gg += 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
# Try reproducing the correlation between [OIII] and "L_B" given in Zakamska+2003
for gal in gals:
    
    # Open the FITS file containing the extracted spectrum
    fname = os.path.join(fits_path, f"{gal}_COMB_Re.fits")
    if os.path.exists(fname):
        hdulist = fits.open(fname)
    else:
        continue
    spec = hdulist[0].data
    spec_err = hdulist[1].data
    dlambda_A = hdulist[0].header["CDELT1"]
    lambda_vals_A = np.array(range(hdulist[0].header["NAXIS1"])) + hdulist[0].header["CRVAL1"] * dlambda_A
    
    # Compute the rest-frame L_B (eqn. 6)
    z = hdulist[0].header["Z"]
    lambda_vals_A /= (1 + z)
    dlambda_A /= (1 + z)
    lambda_start_idx = np.nanargmin(np.abs(lambda_vals_A - 3980))
    lambda_stop_idx = np.nanargmin(np.abs(lambda_vals_A - 4920))
    L_B = np.nansum(spec[lambda_start_idx:lambda_stop_idx] * dlambda_A)
    
    df_s7.loc[gal, "L_B"] = L_B
    df_s7.loc[gal, "log L_B"] = np.log10(L_B)


In [118]:
# plot: is there a strong correlation?
fig, ax = plt.subplots()
ax.scatter(x=df_s7["log L_B"], y=df_s7["log L_OIII5007 (total)"])
fn_Z03 = lambda log_L_B: 1.15 * log_L_B - 3.4
ax.plot([40, 43], [fn_Z03(40), fn_Z03(43)])
ax.set_xlabel("log L_B")
ax.set_ylabel("log L_OIII5007")
ax.grid()
ax.autoscale(tight=True)


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …